In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score, recall_score
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#df = pd.read_csv('asthma_dataset_0 (1).csv')
df = pd.read_csv("C:\\Users\\aysen\\Downloads\\asthma_dataset_0 (2).csv")

#df = df.drop(['Unnamed: 0'], axis=1)
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)


print(df.shape)
print("Number of non-null rows in EventDay: ", df.EventDay.count())
display(df.head())

(2575, 187)
Number of non-null rows in EventDay:  200


,SubjectNr,DayNo,weekday,dayType,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,school_yes_no,activity_score_child,screentime,sleep_score_child,bedtimeReport,waketimeReport,fvc_best,fev1_best,pef_best,grade_fev1,grade_fvc,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,BASELINE_PedsQL_score,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,AllergicRhin,AtopicAsthma,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Eosinophils,ExerciseSymp,Family_his,Gender,Height,Hosp_treatme,Hospital_Trt,ICS,LABA,Leukotri_Mod,Monoclonals,Oral_Steroid,OtherClass,PancrInsuffi,PetsHome,Pseudomonas,SABA,SchoolYear,Serum_IgE,SmokeHome,Sports,Vaccination,Weight,school_year_edit,Urbanisation,urbanisation2,totalSleepDuration,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay
0,1771258,0,Tue,school,3723.0,3723.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322.4,461.2,63.0,137.0,274.2,341.2,605.0,1640.0,376.4,82.0,451.0,245.0,135.0,160.0,164.0,41.0,0.0,68.0,120.0,8.689362e+09,78.42,73.22,70.71,67.9,68.74,68.54,69.37,87.78,99.44,101.38,99.36,120.0,103.26,103.88,93.28,101.70,88.76,89.8,83.4,87.8,91.5,93.30,77.1,78.6,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3232.0,2381.0,4334.00,D,B,3.0,0.736696,0.726555,0.877857,0.822316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,NaN,1,15,7.0,No,Yes,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,NaN,No,No,Female,163.2,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,4.0,1465.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33240.0,NaN,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN
1,1771258,1,Wed,school,10015.0,10015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339.2,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,0.0,82.0,168.0,1.326242e+09,73.60,75.20,70.75,92.0,76.00,70.30,82.30,78.00,99.00,113.00,129.80,95.1,91.60,97.02,108.50,85.25,93.40,95.8,101.8,142.3,107.8,104.20,91.8,93.0,180.0,22200.0,11760.0,0.0,84.0,67.0,121.0,8.0,Ja,8.0,B. 30 tot 60 minuten,9.0,15:34:00,13:04:00,3261.0,2257.0,4509.00,A,A,3.0,0.692119,0.688716,0.885734,0.772559,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.500000,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,10.0,12,8,11.3,100,13.1,0.2,2,0.9,2,1,3.0,3.47,32.18,37.49,25.72,29.69,10.83,3.09,NaN,1,15,7.0,No,Yes,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,NaN,No,No,Female,163.2,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,4.0,1465.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extrem

In [3]:
def preprocess_for_models(df, feature_selection, threshold):
    
    # Drop columns that have NaN except for EventDay column
    columns_to_drop = df.columns[df.isna().any() & ~df.columns.isin(['EventDay'])]
    df_cleaned = df.drop(columns=columns_to_drop)
    
    # OHE
    one_hot_asthma = pd.get_dummies(df_cleaned[df_cleaned.columns[df_cleaned.dtypes == object].tolist()])
    asthma_df = pd.concat([df_cleaned, one_hot_asthma], axis=1)
    asthma_df = asthma_df.drop(columns=df_cleaned.columns[df_cleaned.dtypes == object].tolist())
    
    df_in_use = asthma_df
    
    # Variance Threshold feature selection
    if feature_selection:
        columns_to_drop = ['EventDay', 'SubjectNr', 'DayNo']
        selected_features = asthma_df.columns.drop(columns_to_drop)
        selector = VarianceThreshold(threshold)
        selector.fit(asthma_df[selected_features])
        selected_features = selected_features[selector.get_support()]

        df_selected = asthma_df[selected_features]
        df_selected = pd.concat([asthma_df[columns_to_drop], df_selected], axis=1)
        df_in_use = df_selected
        
    # df without eventDay column
    df_withoutEventDay = df_in_use[df_in_use['EventDay'].isnull()]
    
    # Remove useless columns & Scale
    training = df_withoutEventDay.drop(['EventDay', 'SubjectNr', 'DayNo'], axis = 1)   # without eventDay rows
    predicting = df_in_use.drop(['EventDay', 'SubjectNr', 'DayNo'], axis = 1)          # with eventDay rows

    # Scale training
    data_train = training
    scaler_train = StandardScaler()
    np_scaled_train = scaler_train.fit_transform(data_train)
    data_train = pd.DataFrame(np_scaled_train, columns=training.columns)

    # Scale predicting
    data_pred = predicting
    scaler_pred = StandardScaler()
    np_scaled_pred = scaler_pred.fit_transform(data_pred)
    data_pred = pd.DataFrame(np_scaled_pred, columns=predicting.columns)
    
    return data_train, data_pred, asthma_df, np_scaled_train, np_scaled_pred

In [4]:
# asthma_df is needed only in k-Means when I add back the other columns
# np_scaled_train and np_scaled_ored are only needed in the DL models
data_train, data_pred, asthma_df, np_scaled_train, np_scaled_pred = preprocess_for_models(df, True, 0.2)
print(data_train.shape, data_pred.shape)

(2375, 78) (2575, 78)


# k-Means Cluster

In [18]:
def suggested_no_of_clusters(df, min_clusters, max_clusters):
    # Initialize lists to store the number of clusters and corresponding WCSS values
    num_clusters = []
    wcss_values = []

    # Perform clustering for different numbers of clusters
    for k in range(min_clusters, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(df)

        # Compute the within-cluster sum of squares (WCSS)
        wcss = kmeans.inertia_

        # Append the number of clusters and WCSS values to the lists
        num_clusters.append(k)
        wcss_values.append(wcss)

    # Calculate the differences between consecutive WCSS values
    wcss_diff = [wcss_values[i] - wcss_values[i-1] for i in range(1, len(wcss_values))]

    # Find the index of the maximum difference
    max_diff_index = wcss_diff.index(max(wcss_diff))

    # Suggested number of clusters
    suggested_clusters = num_clusters[max_diff_index]

    print("Suggested number of clusters:", suggested_clusters)
    
suggested_no_of_clusters(data_train, 1, 10)

Suggested number of clusters: 9


In [31]:
def k_means(data_train, data_pred, num_clusters, threshold):
    # Select relevant columns for clustering
    columns_to_cluster = data_train.columns

    # Extract the subset of data for clustering from training DataFrame
    data_for_clustering = data_train[columns_to_cluster]

    # Standardize the data
    scaler = StandardScaler()
    data_for_clustering_standardized = scaler.fit_transform(data_for_clustering)

    # Apply k-means clustering on training data
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(data_for_clustering_standardized)

    # Get the cluster labels for training data
    training_cluster_labels = kmeans.labels_

    # Assign cluster labels to the training DataFrame
    data_train['ClusterLabel'] = training_cluster_labels

    # Check the distribution of clusters in training data
    training_cluster_counts = data_train['ClusterLabel'].value_counts()

    # Calculate cluster means on training data
    training_cluster_means = data_train.groupby('ClusterLabel').mean()



    # Extract the subset of data for anomaly prediction from predicting DataFrame
    data_for_prediction = data_pred[columns_to_cluster]

    # Standardize the data for prediction using the same scaler as training
    data_for_prediction_standardized = scaler.transform(data_for_prediction)

    # Get the cluster labels for prediction data
    prediction_cluster_labels = kmeans.predict(data_for_prediction_standardized)

    # Assign cluster labels to the predicting DataFrame
    data_pred['ClusterLabel'] = prediction_cluster_labels

    # Check the distribution of clusters in predicting data
    prediction_cluster_counts = data_pred['ClusterLabel'].value_counts()

    # Detect anomalies based on cluster means
    anomaly_rows = []
    for index, row in data_pred.iterrows():
        cluster_label = row['ClusterLabel']
        features = row[columns_to_cluster]
        cluster_mean = training_cluster_means.loc[cluster_label]
        if any(abs(features - cluster_mean) > threshold):
            anomaly_rows.append(index)

    # detected anomalies
    anomalies = data_pred.loc[anomaly_rows]
    
    

    # Add back the other columns
    corresponding_info = asthma_df.loc[anomalies.index, ['EventDay', 'SubjectNr', 'DayNo']]
    anomalies_with_info = pd.concat([anomalies, corresponding_info], axis=1)

    print("Total amount of anomalies detected:            ",len(anomalies_with_info))
    print("Amount of non-null rows in EventDay:           ", anomalies_with_info['EventDay'].count())

    # Count the non-null values in the 'EventDay' column
    non_null_count = anomalies_with_info['EventDay'].count()

    # Calculate the total number of rows
    total_rows = len(anomalies_with_info)

    # Calculate the percentage
    percentage = (non_null_count / total_rows) * 100

    # Print the percentage
    print("Percentage of non-null rows in EventDay:       ", percentage, "%")

In [32]:
k_means(data_train, data_pred, 5, 10)

Total amount of anomalies detected:             43
Amount of non-null rows in EventDay:            2
Percentage of non-null rows in EventDay:        4.651162790697675 %


# Isolation Forest

In [49]:
def iforest(data_train, data_pred, outliers_fraction):
    # Train
    ifo = IsolationForest(contamination = outliers_fraction)
    ifo.fit(data_train)

    # Predict
    df['anomaly1'] = pd.Series(ifo.predict(data_pred))
    
    # Prints
    non_null_count = df[df.anomaly1 == -1]['EventDay'].count()
    total_rows = df.EventDay.count()
    percentage = (non_null_count / total_rows) * 100
    print("Number of rows labelled as outliers:           ", df.anomaly1.value_counts()[-1], "out of", len(df))
    print("Amount of non-null rows in EventDay:           ", non_null_count, "out of 200")
    print("Percentage of non-null rows in EventDay:       ", percentage, "%")

In [64]:
iforest(data_train, data_pred, 0.5)

Number of rows labelled as outliers:            1304 out of  2575
Amount of non-null rows in EventDay:            120 out of 200
Percentage of non-null rows in EventDay:        60.0 %


# OSVM

In [65]:
def ocsvm(data_train, data_pred, outliers_fraction):
    # Train
    osvm = OneClassSVM(nu = outliers_fraction, kernel = 'sigmoid')
    osvm.fit(data_train)

    # Predict
    df['anomaly2'] = pd.Series(osvm.predict(data_pred))
    
    # Prints
    non_null_count = df[df.anomaly2 == -1]['EventDay'].count()
    total_rows = df.EventDay.count()
    percentage = (non_null_count / total_rows) * 100
    print("Number of rows labelled as outliers:           ", df.anomaly2.value_counts()[-1], "out of", len(df))
    print("Amount of non-null rows in EventDay:           ", non_null_count, "out of 200")
    print("Percentage of non-null rows in EventDay:       ", percentage, "%")

In [66]:
ocsvm(data_train, data_pred, 0.9)

Number of rows labelled as outliers:            2305 out of 2575
Amount of non-null rows in EventDay:            176 out of 200
Percentage of non-null rows in EventDay:        88.0 %


# Local Outlier Factor

In [82]:
def local_of(data_train, data_pred, outliers_fraction):
    # Train
    lof = LocalOutlierFactor(contamination=outliers_fraction)
    lof.fit(data_train)

    # Predict
    df['anomaly3'] = pd.Series(lof.fit_predict(data_pred))

    # Prints
    non_null_count = df[df.anomaly3 == -1]['EventDay'].count()
    total_rows = df.EventDay.count()
    percentage = (non_null_count / total_rows) * 100
    print("Number of rows labelled as outliers:           ", df.anomaly3.value_counts()[-1], "out of", len(df))
    print("Amount of non-null rows in EventDay:           ", non_null_count, "out of 200")
    print("Percentage of non-null rows in EventDay:       ", percentage, "%")

In [83]:
local_of(data_train, data_pred, 0.3)

Number of rows labelled as outliers:            773 out of 2575
Amount of non-null rows in EventDay:            60 out of 200
Percentage of non-null rows in EventDay:        30.0 %


# Autoencoders

In [92]:
def autoencoders(np_scaled_train, np_scaled_pred):

    # Define the architecture of the autoencoder
    input_dim = np_scaled_train.shape[1]
    encoding_dim = 8  # Set the dimensionality of the bottleneck layer

    input_layer = Input(shape=(input_dim,))
    encoder = Dense(32, activation='sigmoid')(input_layer)
    encoder = Dense(encoding_dim, activation='sigmoid')(encoder)  
    decoder = Dense(32, activation='sigmoid')(encoder)
    decoder = Dense(input_dim, activation='linear')(decoder)

    # Create the autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoder)

    # Compile the model, prevent overfitting and train
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae')
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    autoencoder.fit(np_scaled_train, np_scaled_train, epochs=100, batch_size=64, shuffle=True, validation_split=0.2)

    # Use the trained autoencoder to reconstruct the data points
    reconstructions = autoencoder.predict(np_scaled_pred)

    # Calculate the reconstruction error for each data point
    mse = np.mean(np.power(np_scaled_pred - reconstructions, 2), axis=1)

    # Set a threshold for the reconstruction error above which a data point is considered an anomaly
    threshold = np.percentile(mse, 99)

    # Flag data points with reconstruction errors above the threshold as anomalies
    anomalies = data_pred[mse > threshold]

    # Create a new column 'anomaly4' in the DataFrame to indicate anomalies
    df['anomaly4'] = np.where(mse > threshold, -1, 1)

    non_null_count = df[df.anomaly4 == -1]['EventDay'].count()
    total_rows = df.EventDay.count()
    percentage = (non_null_count / total_rows) * 100
    print("Number of rows labelled as outliers:           ", df.anomaly4.value_counts()[-1], "out of", len(df))
    print("Amount of non-null rows in EventDay:           ", non_null_count, "out of 200")
    print("Percentage of non-null rows in EventDay:       ", percentage, "%")

In [93]:
autoencoders(np_scaled_train, np_scaled_pred)

Epoch 1/100
30/30 [==============================] - 2s 12ms/step - loss: 0.7088 - val_loss: 0.6672
Epoch 2/100
30/30 [==============================] - 0s 5ms/step - loss: 0.6719 - val_loss: 0.6611
Epoch 3/100
30/30 [==============================] - 0s 6ms/step - loss: 0.6671 - val_loss: 0.6538
Epoch 4/100
30/30 [==============================] - 0s 9ms/step - loss: 0.6658 - val_loss: 0.6504
Epoch 5/100
30/30 [==============================] - 0s 5ms/step - loss: 0.6642 - val_loss: 0.6482
Epoch 6/100
30/30 [==============================] - 0s 5ms/step - loss: 0.6618 - val_loss: 0.6450
Epoch 7/100
30/30 [==============================] - 0s 5ms/step - loss: 0.6577 - val_loss: 0.6384
Epoch 8/100
30/30 [==============================] - 0s 4ms/step - loss: 0.6514 - val_loss: 0.6308
Epoch 9/100
30/30 [==============================] - 0s 4ms/step - loss: 0.6435 - val_loss: 0.6210
Epoch 10/100
30/30 [==============================] - 0s 9ms/step - loss: 0.6358 - val_loss: 0.6148
Epoch 11

Epoch 83/100
30/30 [==============================] - 0s 5ms/step - loss: 0.4978 - val_loss: 0.5878
Epoch 84/100
30/30 [==============================] - 0s 6ms/step - loss: 0.4967 - val_loss: 0.5876
Epoch 85/100
30/30 [==============================] - 0s 5ms/step - loss: 0.4954 - val_loss: 0.5867
Epoch 86/100
30/30 [==============================] - 0s 8ms/step - loss: 0.4942 - val_loss: 0.5843
Epoch 87/100
30/30 [==============================] - 0s 5ms/step - loss: 0.4932 - val_loss: 0.5845
Epoch 88/100
30/30 [==============================] - 0s 6ms/step - loss: 0.4920 - val_loss: 0.5856
Epoch 89/100
30/30 [==============================] - 0s 7ms/step - loss: 0.4911 - val_loss: 0.5824
Epoch 90/100
30/30 [==============================] - 0s 8ms/step - loss: 0.4899 - val_loss: 0.5827
Epoch 91/100
30/30 [==============================] - 0s 5ms/step - loss: 0.4888 - val_loss: 0.5832
Epoch 92/100
30/30 [==============================] - 0s 5ms/step - loss: 0.4878 - val_loss: 0.5818


# Deep Support Vector Data Description

In [5]:
def dsvdd(data_train, data_pred):
    # Convert the preprocessed data to a PyTorch tensor
    tensor_data = torch.tensor(data_train.values, dtype=torch.float32)
    tensor_data_pred = torch.tensor(data_pred.values, dtype=torch.float32)

    # Create a DataLoader for efficient batch processing
    batch_size = 64
    data_loader = DataLoader(TensorDataset(tensor_data), batch_size=batch_size, shuffle=True)

    # Define the DSVDD model
    class Autoencoder(nn.Module):
        def __init__(self, input_dim, hidden_dim):
            super(Autoencoder, self).__init__()
            self.encoder = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(True),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(True),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(True)
            )
            self.decoder = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(True),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(True),
                nn.Linear(hidden_dim, input_dim),
                nn.Tanh()
            )

        def forward(self, x):
            encoded = self.encoder(x)
            decoded = self.decoder(encoded)
            return encoded, decoded

    # Train the DSVDD model
    input_dim = data_train.shape[1]
    hidden_dim = 64
    num_epochs = 50
    learning_rate = 0.001

    autoencoder = Autoencoder(input_dim, hidden_dim)
    optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

    autoencoder.train()
    for epoch in range(num_epochs):
        for batch_idx, data_batch in enumerate(data_loader):
            optimizer.zero_grad()
            inputs = data_batch[0]

            encoded, decoded = autoencoder(inputs)

            # Reconstruction loss
            reconstruction_loss = nn.MSELoss()(decoded, inputs)

            loss = reconstruction_loss

            loss.backward()
            optimizer.step()

    # Extract the learned features
    autoencoder.eval()
    with torch.no_grad():
        encoded_data, _ = autoencoder(tensor_data)
        encoded_data_pred, _ = autoencoder(tensor_data_pred)

    # Train an SVM/OSVM using the learned features
    svm = OneClassSVM(kernel='sigmoid', nu=0.9)  # Adjust the hyperparameters as needed
    svm.fit(encoded_data.detach().numpy())

    # Detect anomalies
    anomalies = df[svm.predict(encoded_data_pred.detach().numpy()) == -1]  # Filter the original DataFrame for anomalies

    df['anomaly5'] = 1
    df.loc[anomalies.index, 'anomaly5'] = -1

    non_null_count = df[df.anomaly5 == -1]['EventDay'].count()
    total_rows = df.EventDay.count()
    percentage = (non_null_count / total_rows) * 100
    print("Number of rows labelled as outliers:           ", df.anomaly5.value_counts()[-1], "out of", len(df))
    print("Amount of non-null rows in EventDay:           ", non_null_count, "out of 200")
    print("Percentage of non-null rows in EventDay:       ", percentage, "%")

In [6]:
dsvdd(data_train, data_pred)

Number of rows labelled as outliers:            2314 out of 2575
Amount of non-null rows in EventDay:            176 out of 200
Percentage of non-null rows in EventDay:        88.0 %


## What rows did all agree on

In [130]:
m1 = df.anomaly1 == -1 # Isolation Forest
m2 = df.anomaly2 == -1 # OSVM
m3 = df.anomaly3 == -1 # LOF
m4 = df.anomaly4 == -1 # Autoencoders
m5 = df.anomaly5 == -1 # DSVDD

In [136]:
logic = m1 & m3 & m4 & m5
print("Length:",len(df[logic]) , ", EventDay rows:",df[logic].EventDay.count())
df[logic]

Length: 26 , EventDay rows: 1


,SubjectNr,DayNo,weekday,dayType,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,school_yes_no,activity_score_child,screentime,sleep_score_child,bedtimeReport,waketimeReport,fvc_best,fev1_best,pef_best,grade_fev1,grade_fvc,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,BASELINE_PedsQL_score,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,AllergicRhin,AtopicAsthma,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Eosinophils,ExerciseSymp,Family_his,Gender,Height,Hosp_treatme,Hospital_Trt,ICS,LABA,Leukotri_Mod,Monoclonals,Oral_Steroid,OtherClass,PancrInsuffi,PetsHome,Pseudomonas,SABA,SchoolYear,Serum_IgE,SmokeHome,Sports,Vaccination,Weight,school_year_edit,Urbanisation,urbanisation2,totalSleepDuration,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay,anomaly1,anomaly2,anomaly3,anomaly4,anomaly5
84,8380204,0,Fri,school,14744.0,14725.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.6,1036.6,17.0,708.6,266.0,917.0,508.0,1237.0,3041.0,2218.0,86.0,50.0,778.0,1260.0,1919.0,874.0,1554.0,73.0,140.0,9.839726e+09,76.56,75.80,78.60,74.12,73.94,74.24,81.80,84.26,87.06,92.00,99.44,96.00,97.20,100.80,89.80,124.70,100.00,78.80,99.25,92.20,94.00,95.00,101.30,108.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2616.0,2076.00,3833.00,C,C,3.0,0.793578,9.881159e-01,1.066489e+08,0.921372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.333333e+09,3.166667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.913043e+09,5.260870e+09,5.4,4.2,5.5,5.1,6.75,6125.0,7.717391e+09,6.521739e+08,5.1,7,3,4.2,14,7.2,1.0,12,0.0,0,0,0.0,46.53,43.36,114.70,1.21,23.28,33.12,24.28,NaN,0,10,5.0,Yes,Yes,7,2008.0,24.9,3.2,Controlled asthma,school,0.0,0.0,0.17,No,Yes,Male,142.5,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,7.0,NaN,No,Yes,0.0,50.7,Primary school year 7,1.0,Extremely urbanised,NaN,5395.0,1.0,3041.0,98.0,97.0,NaN,Controlled asthma,NaN,-1,-1,-1,-1,-1
100,8380204,16,Sun,weekend,7528.0,7519.0,575.0,639.0,90.0,0.0,0.0,0.0,0.0,52.0,85.0,203.0,2028.0,206.0,597.0,301.0,325.0,444.0,168.0,906.0,599.0,158.0,143.0,230.2,0.0,0.0,62.0,112.0,8.681879e+09,100.10,93.40,91.60,78.70,71.60,74.10,76.00,83.40,85.80,92.80,100.20,101.70,93.10,80.10,90.50,92.80,82.10,105.25,91.70,79.00,95.80,65.10,78.30,70.60,1140.0,22081.0,14759.0,3.0,75.0,65.0,106.0,5.0,Nee,4.0,E. Meer dan 4 uur,3.0,18:49:00,13:19:00,2361.0,1864.00,4422.00,E,E,3.0,0.789496,8.865682e-01,9.618614e-01,0.916657,4.0,3.0,1.0,1.0,1.0,3.0,2.0,2.000000e+00,1.333333e+09,3.166667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.913043e+09,5.260870e+09,5.4,4.2,5.5,5.1,6.75,6125.0,7.717391e+09,6.521739e+08,13.9,16,12,8.6,59,9.9,2.2,28,5.1,55,13,1.0,2.04,6.84,9.97,62.22,36.44,23.63,4.40,NaN,0,10,5.0,Yes,Yes,7,2008.0,24.9,3.2,Controlled asthma,school,0.0,0.0,0.17,No,Yes,Male,142.5,0.